In [1]:
# from google.colab import drive
import os

# drive.mount('/content/drive/')
# os.chdir('/content/drive/My Drive/BERT/SA')

In [2]:
BATCH_SIZE = 16

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
os.chdir('/kaggle/working/')

In [5]:
!nvidia-smi

Sun Jun  2 05:45:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
!pip install transformers

In [7]:
# !pip install fastBPE
# !pip install fairseq

In [8]:
# Install the vncorenlp python wrapper
!pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter)
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645932 sha256=67ce3c1c529cb7e8490210b03781bc4978efff1e4f0047c361ea5540efd67a38
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp
--2024-06-02 05:45:35--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: 'VnCoreNLP-1.1.1.jar'

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M   167MB/s    in 0.2s    

2024-06-02 05:45:37 (167 MB/s) - 'VnCor

In [9]:
!wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
!tar -xzvf PhoBERT_base_fairseq.tar.gz

--2024-06-02 05:45:43--  https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
Resolving public.vinai.io (public.vinai.io)... 18.64.174.104, 18.64.174.107, 18.64.174.113, ...
Connecting to public.vinai.io (public.vinai.io)|18.64.174.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1243308020 (1.2G) [application/x-tar]
Saving to: 'PhoBERT_base_fairseq.tar.gz'

PhoBERT_base_fairse 100%[===================>]   1.16G   147MB/s    in 8.5s    

2024-06-02 05:45:52 (139 MB/s) - 'PhoBERT_base_fairseq.tar.gz' saved [1243308020/1243308020]

PhoBERT_base_fairseq/
PhoBERT_base_fairseq/bpe.codes
PhoBERT_base_fairseq/model.pt
PhoBERT_base_fairseq/dict.txt


In [10]:
!wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
!tar -xzvf PhoBERT_base_transformers.tar.gz


--2024-06-02 05:46:12--  https://public.vinai.io/PhoBERT_base_transformers.tar.gz
Resolving public.vinai.io (public.vinai.io)... 18.64.174.104, 18.64.174.107, 18.64.174.113, ...
Connecting to public.vinai.io (public.vinai.io)|18.64.174.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322405979 (307M) [application/x-tar]
Saving to: 'PhoBERT_base_transformers.tar.gz'

PhoBERT_base_transf 100%[===================>] 307.47M   123MB/s    in 2.5s    

2024-06-02 05:46:15 (123 MB/s) - 'PhoBERT_base_transformers.tar.gz' saved [322405979/322405979]

PhoBERT_base_transformers/
PhoBERT_base_transformers/config.json
PhoBERT_base_transformers/bpe.codes
PhoBERT_base_transformers/model.bin
PhoBERT_base_transformers/dict.txt


In [11]:
!pip install vncorenlp


In [12]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("/kaggle/working/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

text = "Đại học Bách Khoa Hà Nội."

word_segmented_text = rdrsegmenter.tokenize(text)
print(word_segmented_text)

[['Đại_học', 'Bách_Khoa', 'Hà_Nội', '.']]


In [13]:
# from fairseq.data.encoders.fastbpe import fastBPE
# from fairseq.data import Dictionary
# import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument('--bpe-codes',
#     default="/content/drive/My Drive/PhoBERT_base_transformers/bpe.codes",
#     required=False,
#     type=str,
#     help='path to fastBPE BPE'
# )
# args, unknown = parser.parse_known_args()
# bpe = fastBPE(args)

# # Load the dictionary
# vocab = Dictionary()
# vocab.add_from_file("/content/drive/My Drive/PhoBERT_base_transformers/dict.txt")

In [14]:
# bpe.encode("Hôm_nay trời nóng quá nên tôi ở nhà viết Viblo!")

In [15]:
# vocab.encode_line('<s> ' + 'Hôm_nay trời nóng quá nên tôi ở nhà viết Vi@@ blo@@ !' + ' </s>')

In [16]:
# import re
# import pandas as pd

# train_path = 'https://raw.githubusercontent.com/lavibula/SentimentAnalysis-with-Vietnamese-reviews/main/absa_data/train.csv'
# test_path = 'https://raw.githubusercontent.com/lavibula/SentimentAnalysis-with-Vietnamese-reviews/main/absa_data/test.csv'

# train_set = pd.read_csv(train_path)
# test_set = pd.read_csv(test_path)

# train_set['Sentence'] = train_set['Sentence'].apply(rdrsegmenter.tokenize).apply(lambda x: ' '.join(x[0]))
# test_set['Sentence'] = test_set['Sentence'].apply(rdrsegmenter.tokenize).apply(lambda x: ' '.join(x[0]))

# train_set

In [17]:
import re
import pandas as pd

train_path = 'https://raw.githubusercontent.com/lavibula/SentimentAnalysis-with-Vietnamese-reviews/main/absa_data/train.csv'
test_path = 'https://raw.githubusercontent.com/lavibula/SentimentAnalysis-with-Vietnamese-reviews/main/absa_data/test.csv'

train_set = pd.read_csv(train_path)
test_set = pd.read_csv(test_path)

train_set['Sentence'] = train_set['Sentence'].apply(rdrsegmenter.tokenize).apply(lambda x: ' '.join(x[0]))
test_set['Sentence'] = test_set['Sentence'].apply(rdrsegmenter.tokenize).apply(lambda x: ' '.join(x[0]))

df = pd.concat([train_set, test_set]).reset_index().drop(columns = 'index')
df

,Sentence,FACILITY,LECTURER,OTHERS,TRAINING_PROGRAM
0,slide giáo_trình đầy_đủ,0,0,0,1
1,nhiệt_tình giảng_dạy gần_gũi với sinh_viên,0,1,0,0
2,đi học đầy_đủ full điểm chuyên_cần,0,0,0,2
3,chưa áp_dụng công_nghệ_thông_tin và các thiết_...,0,2,0,0
4,thầy giảng bài hay có nhiều bài_tập ví_dụ ngay...,0,1,0,0
...,...,...,...,...,...
14587,các slide khó hiểu ngôn_ngữ trong slide phức_t...,0,2,0,0
14588,giáo_viên giảng_dạy có tâm_huyết,0,1,0,0
14589,chia_sẻ cho em nhiều điều hay,0,1,0,0
14590,em tiếp_thu chậm,0,2,0,0


In [18]:
train_set.columns

Index(['Sentence', 'FACILITY', 'LECTURER', 'OTHERS', 'TRAINING_PROGRAM'], dtype='object')

In [19]:
train_set[['FACILITY', 'LECTURER', 'OTHERS', 'TRAINING_PROGRAM']].values

array([[0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 0, 0, 2],
       ...,
       [0, 2, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0]])

In [20]:
import torch
import numpy as np

labels = np.array([0, 2, 0, 1]) # Example labels
def one_hot_encode(labels):
    num_classes = 4

    one_hot_labels = np.zeros((len(labels), num_classes))
    one_hot_labels[np.arange(len(labels)), labels] = 1

    one_hot_labels_tensor = torch.tensor(one_hot_labels, dtype=torch.long)
    return one_hot_labels_tensor
one_hot_encode(labels)

tensor([[1, 0, 0, 0],
        [0, 0, 1, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0]])

In [21]:
train_set

,Sentence,FACILITY,LECTURER,OTHERS,TRAINING_PROGRAM
0,slide giáo_trình đầy_đủ,0,0,0,1
1,nhiệt_tình giảng_dạy gần_gũi với sinh_viên,0,1,0,0
2,đi học đầy_đủ full điểm chuyên_cần,0,0,0,2
3,chưa áp_dụng công_nghệ_thông_tin và các thiết_...,0,2,0,0
4,thầy giảng bài hay có nhiều bài_tập ví_dụ ngay...,0,1,0,0
...,...,...,...,...,...
11421,chỉ vì môn game mà em học hai lần mà không qua...,0,0,0,2
11422,em cảm_ơn cô nhiều,0,1,0,0
11423,giao bài_tập quá nhiều,0,2,0,0
11424,giáo_viên dạy dễ hiểu nhiệt_tình,0,1,0,0


In [22]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Sample data for illustration
# data = train_set[['FACILITY', 'LECTURER', 'OTHERS', 'TRAINING_PROGRAM']].copy()
data = df.copy()

# Create DataFrame
train_set = pd.DataFrame(data)

# Select specific columns
selected_columns = ['FACILITY', 'LECTURER', 'OTHERS', 'TRAINING_PROGRAM']
data_to_encode = train_set[selected_columns]

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
encoded_data = encoder.fit_transform(data_to_encode)

# Convert the result to a DataFrame for better readability
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(selected_columns))

# If you want to list the values
encoded_values_list = encoded_df.values
# print(encoded_values_list)

In [23]:
encoded_df

,FACILITY_0,FACILITY_1,FACILITY_2,FACILITY_3,LECTURER_0,LECTURER_1,LECTURER_2,LECTURER_3,OTHERS_0,OTHERS_1,OTHERS_2,OTHERS_3,TRAINING_PROGRAM_0,TRAINING_PROGRAM_1,TRAINING_PROGRAM_2,TRAINING_PROGRAM_3
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14587,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
14588,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
14589,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
14590,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [24]:
encoded_values_list

array([[1., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.]])

In [25]:
encoded_values_list.shape

(14592, 16)

In [26]:
# encoded_values_list = encoded_values_list.reshape(-1,4,4)

In [27]:
train_set

,Sentence,FACILITY,LECTURER,OTHERS,TRAINING_PROGRAM
0,slide giáo_trình đầy_đủ,0,0,0,1
1,nhiệt_tình giảng_dạy gần_gũi với sinh_viên,0,1,0,0
2,đi học đầy_đủ full điểm chuyên_cần,0,0,0,2
3,chưa áp_dụng công_nghệ_thông_tin và các thiết_...,0,2,0,0
4,thầy giảng bài hay có nhiều bài_tập ví_dụ ngay...,0,1,0,0
...,...,...,...,...,...
14587,các slide khó hiểu ngôn_ngữ trong slide phức_t...,0,2,0,0
14588,giáo_viên giảng_dạy có tâm_huyết,0,1,0,0
14589,chia_sẻ cho em nhiều điều hay,0,1,0,0
14590,em tiếp_thu chậm,0,2,0,0


In [28]:
import re
import pandas as pd

# train_path = 'https://raw.githubusercontent.com/lavibula/SentimentAnalysis-with-Vietnamese-reviews/main/absa_data/train.csv'
# test_path = 'https://raw.githubusercontent.com/lavibula/SentimentAnalysis-with-Vietnamese-reviews/main/absa_data/test.csv'

# train_set = pd.read_csv(train_path)
# test_set = pd.read_csv(test_path)

train_id, train_text, train_labels = [], [], []
test_id, test_text = [], []

train_id = [('train_' + str(i)) for i in range(train_set.shape[0])]
train_text = list(train_set['Sentence'].values)
train_labels = encoded_values_list


test_id = [('test_' + str(i)) for i in range(test_set.shape[0])]
test_text = list(test_set['Sentence'].values)
# test_labels = encoded_values_list


In [29]:
train_labels

array([[1., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.]])

In [30]:
from sklearn.model_selection import train_test_split

train_sents, val_sents, train_labels, val_labels = train_test_split(train_text, train_labels, test_size=0.1)

In [31]:
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# MAX_LEN = 125

# train_ids = []
# for sent in train_sents:
#     subwords = '<s> ' + bpe.encode(sent) + ' </s>'
#     encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
#     train_ids.append(encoded_sent)

# val_ids = []
# for sent in val_sents:
#     subwords = '<s> ' + bpe.encode(sent) + ' </s>'
#     encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
#     val_ids.append(encoded_sent)

# train_ids = pad_sequences(train_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
# val_ids = pad_sequences(val_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

In [32]:
# train_masks = []
# for sent in train_ids:
#     mask = [int(token_id > 0) for token_id in sent]
#     train_masks.append(mask)

# val_masks = []
# for sent in val_ids:
#     mask = [int(token_id > 0) for token_id in sent]

#     val_masks.append(mask)

In [33]:
import os
import random
import json
import torch
import torch.nn as nn
from functools import partial
from collections import namedtuple
from dataclasses import dataclass, field, asdict
from huggingface_hub import HfApi

# import evaluate
import numpy as np
# from datasets import load_dataset
from transformers import Trainer
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer, TrainingArguments

2024-06-02 05:47:25.072446: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-02 05:47:25.072560: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-02 05:47:25.208410: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [34]:
import torch
from transformers import AutoModel, AutoTokenizer

MODEL_NAME = "vinai/phobert-base-v2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Example input data
texts = train_sents.copy()

# Tokenize input data
train_inputs = tokenizer(
    texts,
    max_length = 256, 
    padding='max_length',
    truncation=True,
    return_tensors="pt"  # Return PyTorch tensors
)
    

# Print the tokenized inputs
print(train_inputs)

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

{'input_ids': tensor([[    0,  1249,    77,  ...,     1,     1,     1],
        [    0, 14460,    15,  ...,     1,     1,     1],
        [    0, 48090,   204,  ...,     1,     1,     1],
        ...,
        [    0,   106,  1236,  ...,     1,     1,     1],
        [    0,   960,   844,  ...,     1,     1,     1],
        [    0,  3857,    15,  ...,     1,     1,     1]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [35]:
# train_ids
# val_ids
# train_labels
# val_labels
# train_masks
# val_masks

In [36]:
# from torch.utils.data import DataLoader

In [37]:
BATCH_SIZE = 16

In [38]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
# train_inputs = torch.tensor(train_ids)
# val_inputs = torch.tensor(val_ids)
train_labels = torch.tensor(train_labels)
# train_masks = torch.tensor(train_masks)
# val_masks = torch.tensor(val_masks)

train_data = TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

In [39]:
import torch
from transformers import AutoModel, AutoTokenizer

MODEL_NAME = "vinai/phobert-base-v2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Example input data
texts = val_sents.copy()

# Tokenize input data
val_inputs = tokenizer(
    texts,
    max_length = 256, 
    padding='max_length',
    truncation=True,
    return_tensors="pt"  # Return PyTorch tensors
)

# Print the tokenized inputs
print(val_inputs)

{'input_ids': tensor([[    0,    90,  6146,  ...,     1,     1,     1],
        [    0, 48090,    50,  ...,     1,     1,     1],
        [    0,  1249,    55,  ...,     1,     1,     1],
        ...,
        [    0,    10,    21,  ...,     1,     1,     1],
        [    0,   116,  2048,  ...,     1,     1,     1],
        [    0,  6146,    10,  ...,     1,     1,     1]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [40]:
val_labels = torch.tensor(val_labels)
val_data = TensorDataset(val_inputs['input_ids'], val_inputs['attention_mask'], val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=BATCH_SIZE)

In [41]:
print(val_inputs['input_ids'].size())
print(val_inputs['attention_mask'].size())
print(val_labels.shape)

torch.Size([1460, 256])
torch.Size([1460, 256])
torch.Size([1460, 16])


In [42]:
# val_labels

In [43]:
# val_labels = torch.tensor(val_labels)
# val_data = TensorDataset(val_inputs['input_ids'], val_inputs['attention_mask'], val_labels)
# val_sampler = SequentialSampler(val_data)
# val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=16)

In [44]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

config = RobertaConfig.from_pretrained(
    "/kaggle/working/PhoBERT_base_transformers/config.json", from_tf=False, num_labels = 16, output_hidden_states=False,
)
BERT_SA = RobertaForSequenceClassification.from_pretrained(
    "/kaggle/working/PhoBERT_base_transformers/model.bin",
    config=config
)

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/PhoBERT_base_transformers/model.bin and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
BERT_SA

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=0)
      (position_embeddings): Embedding(258, 768, padding_idx=0)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [46]:
# import torch.nn as nn
# BERT_SA.classifier.out_proj = nn.Linear(in_features=768, out_features=16, bias=True)

In [47]:
BERT_SA.cuda()
print('Done')

Done


In [48]:
# val_labels[0]

In [49]:
# import numpy as np
# from sklearn.metrics import f1_score, accuracy_score
# import torch

# def flat_accuracy(preds, labels):
#     # Ensure preds is a 2D array (batch_size x num_classes) to use np.argmax correctly
#     # Convert torch tensor to numpy array
#     labels_numpy = labels.numpy()

#     # Flatten the labels to get the actual class indices
#     labels_flat = np.argmax(labels_numpy, axis=1).flatten()
#     if len(preds.shape) == 1:
#         pred_flat = preds
#     else:
#         pred_flat = np.argmax(preds, axis=1).flatten()
        
# #     labels_flat = labels_flat
# #     print(labels_flat, pred_flat)
#     F1_score = f1_score(labels_flat, pred_flat, average='macro')
#     return accuracy_score(labels_flat, pred_flat), F1_score


In [50]:
import wandb

# Log in to wandb programmatically (if not already logged in via CLI)
wandb.login(key="8b8539216f9db012b822d7152559a24eb9f66ac2")

# Initialize a new wandb run
run = wandb.init(project="ABSA-project")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hopmuc123456. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240602_054744-7dopv1ow
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run classic-aardvark-28
wandb: ⭐️ View project at https://wandb.ai/hopmuc123456/ABSA-project
wandb: 🚀 View run at https://wandb.ai/hopmuc123456/ABSA-project/runs/7dopv1ow


In [51]:
import torch
from torch import nn
import torch.nn.functional as F

class CapsuleLoss(nn.Module):

    def __init__(self, smooth=0.1, lamda=0.6):
        super(CapsuleLoss, self).__init__()
        self.smooth = smooth
        self.lamda = lamda

    def forward(self, input, target):
        one_hot = torch.zeros_like(input).to(input.device)
        one_hot = one_hot.scatter(1, target.unsqueeze(-1), 1)
        a = torch.max(torch.zeros_like(input).to(input.device), 1 - self.smooth - input)
        b = torch.max(torch.zeros_like(input).to(input.device), input - self.smooth)
        loss = one_hot * a * a + self.lamda * (1 - one_hot) * b * b
        loss = loss.sum(dim=1, keepdim=False)
        return loss.mean()

# CrossEntropyLoss for Label Smoothing Regularization
class CrossEntropyLoss_LSR(nn.Module):
    def __init__(self, para_LSR=0.2):
        super(CrossEntropyLoss_LSR, self).__init__()
        self.para_LSR = para_LSR
        self.logSoftmax = nn.LogSoftmax(dim=-1)

    def _toOneHot_smooth(self, label, batchsize, classes):
        prob = self.para_LSR * 1.0 / classes
        one_hot_label = torch.zeros(batchsize, classes) + prob
        for i in range(batchsize):
            index = label[i]
            one_hot_label[i, index] += (1.0 - self.para_LSR)
        return one_hot_label

    def forward(self, pre, label, size_average=True):
        b, c = pre.size()
        one_hot_label = self._toOneHot_smooth(label, b, c).to(pre.device)
        loss = torch.sum(-one_hot_label * self.logSoftmax(pre), dim=1)
        if size_average:
            return torch.mean(loss)
        else:
            return torch.sum(loss)

class SmoothCrossEntropy(nn.Module):

    def __init__(self, smooth=0.08):
        super(SmoothCrossEntropy, self).__init__()
        self.kldiv = nn.KLDivLoss()
        self.smooth = smooth

    def forward(self, input, target):
        one_hot = torch.zeros_like(input).to(input.device)
        one_hot = one_hot.scatter(1, target.unsqueeze(-1), 1)
        target = (1 - self.smooth) * one_hot + self.smooth / (input.size(1) - 1) * (1 - one_hot)
        # target = target + torch.rand_like(target).to(target.device) * 0.001
        input = input - input.max(dim=1, keepdim=True)[0]
        loss = -target * F.log_softmax(input, dim=-1)
        return loss.mean()

In [52]:
import random
from tqdm import tqdm_notebook
import torch
import torch.nn.functional as F
from transformers import AdamW

device = 'cuda'
epochs = 10

param_optimizer = list(BERT_SA.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

criterion = CapsuleLoss()
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-6, correct_bias=False)


for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    
    BERT_SA.train()
    train_accuracy = 0
    nb_train_steps = 0
    train_f1 = 0
    total_loss = 0
    total_samples = 0
    correct_samples = 0
    
    for step, batch in tqdm_notebook(enumerate(train_dataloader)):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        BERT_SA.zero_grad()
        outputs = BERT_SA(b_input_ids,
                          token_type_ids=None,
                          attention_mask=b_input_mask,
                          labels=b_labels)

        logits = outputs.logits  # keep this tensor on the GPU
        logits = logits.view(-1, 4, 4)
        label_ids = b_labels.view(-1, 4, 4)
        
        # Convert one-hot encoded labels to class indices
        label_ids_indices = torch.argmax(label_ids, dim=-1)

        # Reshape logits and label_ids
        batch_size, num_capsules, num_classes = logits.shape
        logits_flat = logits.view(batch_size * num_capsules, num_classes)
        label_ids_flat = label_ids_indices.view(batch_size * num_capsules)

        # Calculate loss
        loss = criterion(logits_flat, label_ids_flat)
        total_loss += batch_size * loss.item()
        total_samples += batch_size * num_classes

        pred = logits_flat.argmax(dim=1)
        correct_samples += (label_ids_flat == pred).long().sum().item()
#         print(correct_samples)
#         print(total_samples)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(BERT_SA.parameters(), 1.0)
        optimizer.step()
#         break
        
    train_loss = total_loss / total_samples
    train_accuracy = correct_samples / total_samples

    print(f" Accuracy: {train_accuracy:.4f}")
    print(f" Average training loss: {train_loss:.4f}")

    print("Running Validation...")
    BERT_SA.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    eval_f1 = 0
    total_loss = 0
    total_samples = 0
    correct_samples = 0
    
    for batch in tqdm_notebook(val_dataloader):

        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = BERT_SA(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)
            
            logits = outputs.logits  # keep this tensor on the GPU
            logits = logits.view(-1, 4, 4)
            label_ids = b_labels.view(-1, 4, 4)

            # Convert one-hot encoded labels to class indices
            label_ids_indices = torch.argmax(label_ids, dim=-1)

            # Reshape logits and label_ids
            batch_size, num_capsules, num_classes = logits.shape
            logits_flat = logits.view(batch_size * num_capsules, num_classes)
            label_ids_flat = label_ids_indices.view(batch_size * num_capsules)

            # Calculate loss
            loss = criterion(logits_flat, label_ids_flat)
            total_loss += batch_size * loss.item()
            total_samples += batch_size * num_classes

            pred = logits_flat.argmax(dim=1)
            correct_samples += (label_ids_flat == pred).long().sum().item()
#             break
            
    eval_loss = total_loss / total_samples
    eval_accuracy = correct_samples / total_samples

    print(f" Accuracy: {eval_accuracy:.4f}")
    print(f" Average evaluating loss: {eval_loss:.4f}")
    
    wandb.log({"Train/Accuracy": train_accuracy, 
               "Train/Loss": train_loss,
               "Eval/Accuracy": eval_accuracy,
               "Eval/Loss": eval_loss,
              })

    save_path = f"state_dict_model_{epoch_i + 1}.pt"
    torch.save(BERT_SA.state_dict(), save_path)


======== Epoch 1 / 10 ========
Training...


0it [00:00, ?it/s]

 Accuracy: 0.8252
 Average training loss: 0.0364
Running Validation...


  0%|          | 0/92 [00:00<?, ?it/s]

 Accuracy: 0.8863
 Average evaluating loss: 0.0241
======== Epoch 2 / 10 ========
Training...


...

 Accuracy: 0.9185
 Average evaluating loss: 0.0166
======== Epoch 10 / 10 ========
Training...


0it [00:00, ?it/s]

 Accuracy: 0.9234
 Average training loss: 0.0160
Running Validation...


  0%|          | 0/92 [00:00<?, ?it/s]

 Accuracy: 0.9178
 Average evaluating loss: 0.0165


### Inference

In [53]:
# Specify a path
PATH = save_path

# Save
torch.save(BERT_SA.state_dict(), PATH)

In [54]:
# Load
model = RobertaForSequenceClassification.from_pretrained(
    "/kaggle/working/PhoBERT_base_transformers/model.bin",
    config=config
)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(DEVICE)
model.load_state_dict(torch.load(PATH))
model.eval()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/PhoBERT_base_transformers/model.bin and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=0)
      (position_embeddings): Embedding(258, 768, padding_idx=0)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [55]:
def inference(texts):
    # segmenter
    word_segmented_text = rdrsegmenter.tokenize(text)

    # Tokenize input data
    inputs = tokenizer(
        texts,
        max_length = 256, 
        padding='max_length',
        truncation=True,
        return_tensors="pt"  # Return PyTorch tensors
    )

    inputs.to(DEVICE)
    
    with torch.no_grad():
        outputs = model(inputs['input_ids'],
                    token_type_ids=None,
                    attention_mask=inputs['attention_mask'])

    logits = outputs.logits
    logits = logits.view(-1, 4, 4)
    logits = torch.argmax(logits, dim=-1)[0]

    topic = ['FACILITY', 'LECTURER', 'OTHERS', 'TRAINING_PROGRAM']
    sentiment = ['None', 'Positive', 'Negative', 'Normal']
    for i in range(len(logits)):
        if logits[i] == 0:
            continue
        print('Topic:', topic[i])
        print('Sentiment:', sentiment[logits[i]])

        break
        
inference(["giao bài_tập quá nhiều"])

Topic: LECTURER
Sentiment: Negative


In [56]:
inference(["đi học đầy_đủ full điểm chuyên_cần"])

In [57]:
inference(["chưa áp dụng công nghệ thông tin"])

Topic: LECTURER
Sentiment: Negative


In [58]:
inference(["cô giảng hay dễ hiểu"])

Topic: LECTURER
Sentiment: Positive


In [59]:
inference(["giáo trình cũ không có gì mới"])

Topic: TRAINING_PROGRAM
Sentiment: Negative
